In [1]:
from sklearn.model_selection import train_test_split
#import cv2
import os
import numpy as np
import pandas as pd

import tensorflow as tf


from sklearn.preprocessing import LabelEncoder
import math
from functools import partial
import matplotlib.pyplot as plt
import tqdm
from pandas.core.common import flatten
import itertools

from numpy import unique
from numpy import argmax
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from tensorflow.keras import layers, models

In [2]:
print("GPU Available: ", tf.test.is_gpu_available())
from tensorflow.python.client import device_lib

W0714 07:33:49.851440 140431699314432 deprecation.py:323] From <ipython-input-2-088aa1a5baab>:1: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU Available:  False


In [ ]:
!nvidia-smi

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
os.getcwd()

In [3]:
project_dir = '/Workspace/jiyoon/analysis'
code_dir = os.path.join(project_dir, 'code')
# save_dir = os.path.join(project_dir, 'results')
save_dir = os.path.join(project_dir, 'results')


#X_train = np.load(os.path.join(save_dir, 'X_train_10.npy'))  
#X_test = np.load(os.path.join(save_dir, 'X_test_10.npy'))  
#y_train = np.load(os.path.join(save_dir, 'y_train_10.npy'))  
#y_test = np.load(os.path.join(save_dir, 'y_test_10.npy'))  

X_train = np.load(os.path.join(save_dir, 'X_train_10_total.npy'))  
X_test = np.load(os.path.join(save_dir, 'X_test_10_total.npy'))  
y_train = np.load(os.path.join(save_dir, 'y_train_10_total.npy'))  
y_test = np.load(os.path.join(save_dir, 'y_test_10_total.npy'))  

In [4]:
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train) # angry to 1
y_train = tf.keras.utils.to_categorical(y_train) # 1 to [1,0,0,0,0,0,0]
y_test = labelencoder.transform(y_test) # angry to 1
y_test = tf.keras.utils.to_categorical(y_test) # 1 to [1,0,0,0,0,0,0]

In [5]:
# Normalization
X_train = np.array([np.array(X_train[i], dtype='float32') / 255 for i in range(X_train.shape[0])], dtype='float32')
X_test = np.array([np.array(X_test[i], dtype='float32') / 255 for i in range(X_test.shape[0])], dtype='float32')

# Convert 3D
time_step = 10
X_train = X_train.transpose(0,3,1,2)
X_train_3D = X_train.reshape((-1,time_step,3,108,192))
X_test = X_test.transpose(0,3,1,2)
X_test_3D = X_test.reshape((-1,time_step,3,108,192))
y_train_3D = y_train.reshape((-1,time_step,7))
y_test_3D = y_test.reshape((-1,time_step,7))

In [6]:
model = Sequential()
model.add(Input(shape=X_train_3D[0].shape, name='input'))
model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3), data_format='channels_first', recurrent_activation='hard_sigmoid',
                     activation='tanh', padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_first'))
model.add(ConvLSTM2D(filters=10, kernel_size=(3, 3), data_format='channels_first', padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 3, 3), padding='same', data_format='channels_first'))
model.add(ConvLSTM2D(filters=5, kernel_size=(3, 3), data_format='channels_first', stateful = False,
                     kernel_initializer='random_uniform', padding='same', return_sequences=True))
model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_first'))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512,kernel_initializer='glorot_normal', activation = "relu")))
model.add(TimeDistributed(Dense(y_train_3D.shape[2], kernel_initializer='glorot_normal', activation = "softmax")))

model.compile(tf.keras.optimizers.Adam(learning_rate=1E-4), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
def define_convlstm(X_train, y_train):
    model = Sequential()
    model.add(Input(shape=X_train[0].shape, name='input'))
    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3), data_format='channels_first', recurrent_activation='hard_sigmoid',
                         activation='tanh', padding='same', return_sequences=True))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_first'))
    model.add(ConvLSTM2D(filters=10, kernel_size=(3, 3), data_format='channels_first', padding='same', return_sequences=True))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(1, 3, 3), padding='same', data_format='channels_first'))
    model.add(ConvLSTM2D(filters=5, kernel_size=(3, 3), data_format='channels_first', stateful = False,
                         kernel_initializer='random_uniform', padding='same', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_first'))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(512,kernel_initializer='glorot_normal', activation = "relu")))
    model.add(TimeDistributed(Dense(y_train.shape[2], kernel_initializer='glorot_normal', activation = "softmax")))

    model.compile(tf.keras.optimizers.Adam(learning_rate=1E-4), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [10]:
print(X_train_3D.shape)
print(X_test_3D.shape)
print(y_train_3D.shape)
print(y_test_3D.shape)

(7242, 10, 3, 108, 192)
(3109, 10, 3, 108, 192)
(7242, 10, 7)
(3109, 10, 7)


In [11]:
#early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1E-3, patience=10)           
#history = model.fit(x=X_train, y=y_train, batch_size=16, epochs=100, verbose=2, validation_split=0.2, callbacks=[early_stop]) 

In [ ]:
history = model.fit(x=X_train_3D, y=y_train_3D, batch_size=4, epochs=100)
#history = model.fit(x=X_train, y=y_train, batch_size=128, epochs=100, verbose=2, validation_split=0.2) 

Epoch 1/100
 210/1811 [==>...........................] - ETA: 3:09:05 - loss: 1.9550 - categorical_accuracy: 0.1398

## train results

In [ ]:
train_results = model.predict(x=X_train)
train_results = np.argmax(train_results, axis=1)
print(f'train prediction is {train_results}')

In [ ]:
y_train = np.argmax(y_train, axis=1)
train_acc = accuracy_score(y_train, train_results)
print(f'train accuracy is {train_acc}')

## test results

In [ ]:
test_results = model.predict(x=X_test)
test_results = np.argmax(test_results, axis=1)
print(f'test prediction is {test_results}')

In [ ]:
y_test = np.argmax(y_test, axis=1)
test_acc = accuracy_score(y_test, test_results)
print(f'test accuracy is {test_acc}')

In [ ]:
def plot_confusion_matrix(cm,
                          classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.figure(figsize=(9,9))
    plt.figure(1)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)

    plt.title(title, fontsize=18)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=12)
    plt.yticks(tick_marks, classes, fontsize=12)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=16, labelpad=20)
    plt.xlabel('Predicted label', fontsize=16, labelpad=20)
    
    plt.tight_layout()
    plt.grid(False)
    plt.show(plt.gcf())

In [ ]:
cm = confusion_matrix(y_test, test_results)
plot_confusion_matrix(cm, classes = list(labelencoder.classes_), normalize=False, title = 'confusion matrix')

In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(y_test, test_results, target_names=list(labelencoder.classes_))
print(cr)

In [ ]:
cm = confusion_matrix(y_train, train_results)
plot_confusion_matrix(cm, classes = list(labelencoder.classes_), normalize=False, title = 'confusion matrix')

cr = classification_report(y_train, train_results, target_names=list(labelencoder.classes_))
print(cr)